In [29]:
import importlib
import sys
from tqdm import tqdm
import  os
import glob
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "Osiris//monitoring//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import mplhep as hep
import Timing_tools as TTools
import rawFileReader
import Visual_tools as VTools

hep.style.use([hep.style.ATLAS])

# Specify the directory
data_list = sorted([f for f in os.listdir("data") if os.path.isfile(os.path.join("data", f))], reverse=True) ##all files in data directory sorted from the newest to the oldest

file_path = [dir_path+"//data//"+"proAnubis_240731_0217.raw"] #
file_path = list(map(lambda p: dir_path+"//data//"+p, data_list))[1:] # insert your file

In [17]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
importlib.reload(VTools)

fReader = rawFileReader.fileReader(file_path[0]) # load in the classs object
#fReader.skip_events(10_000_000)
initial_event_chunk = fReader.get_aligned_events(interval=100)
print(RTools.abs_badVsgood_hits_count(initial_event_chunk))

#VTools.time_events(initial_event_chunk)
#VTools.hitHeatMap(initial_event_chunk,0)

[[298, 46], [271, 47], [274, 14], [246, 42], [144, 55]]


In [31]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
import matplotlib.pyplot as plt
interval = 1000 # Set your monitoring chunck size
order = [(0,1), (1,2), (2,3), (3,4)] # Order what you want to align
max_process_event_chunk = 10_000 # End the loop early
actual_mets = {}
for file in file_path:
    file_name = file.split("//")[-1][:-4]
    fReader = rawFileReader.fileReader(file) # reload in the classs object
    processedEvents = 0 # Initialisation

    #Initialise variables to store the results
    mets = [[] for tdc in range(5)]
    empty_header = 0 # Counting the number of empty headers
    emtpy_events_with_header = [[], []] # prepare two lists of same dimensions for line graph plotting
    tdc_event_count = [[[],[]] for tdc in range(5)]# prepare for histogram plotting


    with tqdm(total=max_process_event_chunk, desc=f"Processing {file_name}", unit='Events') as pbar:
        while processedEvents < max_process_event_chunk:
            processedEvents += 1
            event_chunk = fReader.get_aligned_events(order=order, interval=interval) # get the aligned events
            tdc_event_count_buffer = RTools.abs_badVsgood_hits(event_chunk) # finding the number of 
            for tdc in  range(5):
                tdc_event_count[tdc][0].append(tdc_event_count_buffer[tdc][0])
                tdc_event_count[tdc][1].append(tdc_event_count_buffer[tdc][1])
            pbar.update(1)
    
    actual_mets[file_name] = tdc_event_count
    fig, ax = plt.subplots(figsize=(10, 8))
    for tdc in range(4,5):
        met = tdc_event_count[tdc]
        binsx = [x for x in range(len(met[0]))]
        ax.plot(binsx, met[0], label=f'TDC{tdc}')
        ax.plot(binsx, met[1], label=f'TDC{tdc}')

    ax.set_xlim(0, max_process_event_chunk)
    # ax.set_ylim(-1, 100)
    ax.legend()
    ax.set_title('Abs Good vs Bad')
    ax.set_ylabel('num of events')
    ax.set_xlabel('Processed Chunk Events')
    plt.savefig(f"AbsBadVsGood{file_name}")
    

Processing Events: 100%|██████████| 10000/10000 [02:17<00:00, 72.87Events/s]


In [24]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 8))
for tdc in range(4,5):
    met = tdc_event_count[tdc]
    binsx = [x for x in range(len(met[0]))]
    ax.plot(binsx, met[0], label=f'TDC{tdc}')
    ax.plot(binsx, met[1], label=f'TDC{tdc}')

ax.set_xlim(0, max_process_event_chunk)
# ax.set_ylim(-1, 100)
ax.legend()
ax.set_title('TDC number of events recorded')
ax.set_ylabel('num of events')
ax.set_xlabel('Processed Chunk Events')
plt.show()